# 03 Minimal Web API

**autor: Erik Král ekral@utb.cz**

---

V tomto materiálu si probereme práci s webovými službami s pomocí Minimal Web API.

Web API (Web Application Programming Interface) je sada pravidel a protokolů umožňující komunikovat programům prostřednictvím internetu. REST (Representation State Transfer) je druh Web API a představuj architektonický styl použivající standartní HTTP metody (GET, POST, PUT a DELETE) zpřístpňující endpoity identifikvané pomocí URI. Pro přenos dat využívá přitom především format JSON.

Minimal Web API je zjednodušený způsob tvorby HTTP API pomocí ASP.NET Core. 

Nejprve si nainstalujeme nuget pomocí kterého bude moct hostovat Minimal Web API přímo v tomto notebooku.

In [ ]:
#r "nuget:Microsoft.DotNet.Interactive.AspNetCore,*-*"

Installed Packages Microsoft.DotNet.Interactive.AspNetCore, 1.0.0-beta.25110.2

Loading extension script from `C:\Users\erik\.nuget\packages\microsoft.dotnet.interactive.aspnetcore\1.0.0-beta.25110.2\interactive-extensions\dotnet\extension.dib`

Následující kód vrátí na metodu GET "Hello World" spolu s počtem volání této metody. Představuje nejjednodušší program v Minimal Web API.

In [ ]:
#!aspnet

var builder = WebApplication.CreateBuilder();
var app = builder.Build();

int x = 0;
app.MapGet("/", () => $"Hello World {++x}");

await app.StartAsync();


info: Microsoft.Hosting.Lifetime[14]
      Now listening on: http://localhost:5000
info: Microsoft.Hosting.Lifetime[0]
      Application started. Press Ctrl+C to shut down.
info: Microsoft.Hosting.Lifetime[0]
      Hosting environment: Production
info: Microsoft.Hosting.Lifetime[0]
      Content root path: c:\Users\erik\source\repos\FAI\AF\notebooks


Server zastavíme následujícím příkazem:

In [ ]:
await app.StopAsync();